In [1]:
import numpy as np
import pandas as pd
import math
import random

In [4]:
data = pd.read_csv("~/Data/clean_rating4.csv").drop(["id"], axis = 1)
print(len(set(data.anime_id.tolist())))

9775


In [5]:
data = pd.read_csv("~/Data/clean_rating4.csv").drop(["id"], axis = 1)

idx_to_user = list(set(data.user_id))
idx_to_item = list(set(data[data.rating > 0].anime_id))

user_to_idx = {user: idx for idx, user in enumerate(idx_to_user)}
item_to_idx = {item: idx for idx, item in enumerate(idx_to_item)}

ratings = data[data.rating > 0].rating.values.tolist()
data = data[data.rating > 0].values

F, alpha, lam_bda, batch_size = 100, 0.02, 0.01, 512000
num_epochs, n, k = 30, len(ratings), 1 / math.sqrt(F)

P = np.array([[random.random() * k for _ in range(F)] for _ in range(len(idx_to_user))])
Q = np.array([[random.random() * k for _ in range(len(idx_to_item))] for _ in range(F)])

In [6]:
def sigmoid(x):
    return 1.0 / (1 + math.exp(-x))

In [7]:
def train(train_data, alpha, lam_bda):
    index = [i for i in range(train_data.shape[0])]
    for epoch in range(num_epochs):
        sample = random.sample(index, batch_size)
        loss = 0
        for i in range(batch_size):
            user = user_to_idx[train_data[sample[i], 0]]
            item = item_to_idx[train_data[sample[i], 1]]
            rui = train_data[sample[i], 2]

            eui = rui - sigmoid(P[user, :].dot(Q[:, item]))
            loss += eui
            for f in range(F):
                P[user, f] += alpha * (eui * Q[f, item] - lam_bda * P[user, f])
                Q[f, item] += alpha * (eui * P[user, f] - lam_bda * Q[f, item])

        alpha *= 0.9
        print(epoch + 1, loss / batch_size)

In [8]:
def get_hot():
    animes = pd.read_csv("~/Data/anime.csv")
    animes = animes[animes["anime_id"].isin(idx_to_item)].loc[:, ["anime_id", "rating", "members"]].fillna(0)

    scalar = lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))
    animes["rating_norm"] = animes[["rating"]].apply(scalar)
    animes["members_norm"] = animes[["members"]].apply(scalar)
    animes["weight"] = 0.6 * animes["rating_norm"] + 0.4 * animes["members_norm"]
    animes = animes.sort_values(by = "weight", ascending = False)
    
    return animes.anime_id.tolist()

In [9]:
def negative_sample():
    sample_num = 100
    hot_items, neg_sample = get_hot(), []
    data = pd.read_csv("~/Data/clean_rating4.csv")
    grouped = data.groupby(["user_id"])
    for gp in grouped.groups:
        p, num = grouped.get_group(gp).anime_id.tolist(), 0
        for item in hot_items:
            if item not in p: 
                neg_sample.append([gp, item, 0])
                num += 1
            if num == sample_num:
                break

    return np.array(neg_sample)

In [10]:
data[:, -1] = np.array([1 for _ in range(data.shape[0])])
data = np.concatenate((data, negative_sample()))
# from sklearn.model_selection import train_test_split
# train_data, test_data = train_test_split(data, test_size = 0.3)
train_ratio = 0.9
train_data = data[:int(len(data) * train_ratio)]
test_data = data[int(len(data) * train_ratio):]
print(train_data.shape, test_data.shape)

(2411043, 3) (267894, 3)


In [11]:
train(train_data, alpha, lam_bda)

1 0.25563004089640146
2 0.11806794158429655
3 0.06942709650643039
4 0.04950039474304762
5 0.039086621524408005
6 0.03205151822261085
7 0.02744715416219215
8 0.02448388757546437
9 0.02270617323261921
10 0.020554889592266052
11 0.01976336031718647
12 0.019129605543025535
13 0.01898879141959864
14 0.0185896226266003
15 0.01863298857349198
16 0.017962361067556158
17 0.01841639748332317
18 0.018459799323407505
19 0.018153809881136927
20 0.018348595067068773
21 0.01774784219168234
22 0.01851016501879553
23 0.018290887641827507
24 0.018280537468123786
25 0.018629955590590382
26 0.018172467823343388
27 0.01780841010388469
28 0.01778018647983071
29 0.01812357556954971
30 0.01805987877137012


In [25]:
test_users = list(set(test_data[:, 0].tolist()))
df = pd.read_csv("~/Data/clean_rating4.csv").drop(["id"], axis = 1)
user_dict, final, precision = {}, [], []
predict = P.dot(Q)
for k in range(10):
    for user in test_users:
        user_idx = user_to_idx[user]
        pred = predict[user_idx, :]
        dic = {yui: idx for idx, yui in enumerate(pred)}
        topk = [i[1] for i in sorted(dic.items(), key = lambda x: x[0], reverse = True)[1:6]]
        target = [data[1] for data in df[df.user_id == user].values if data[2] > k]
        
#         print(topk, target)
        both = list(set(topk) & set(target))
        precision.append(len(both) / len(topk))

    print(np.mean(precision))
    final.append(np.mean(precision))

0.04088397790055249
0.04077348066298343
0.0407182320441989
0.04055248618784531
0.04022099447513812
0.03968692449355433
0.038587213891081294
0.036505524861878455
0.03378759975445058
0.03087292817679558


In [26]:
print(final)

[0.04088397790055249, 0.04077348066298343, 0.0407182320441989, 0.04055248618784531, 0.04022099447513812, 0.03968692449355433, 0.038587213891081294, 0.036505524861878455, 0.03378759975445058, 0.03087292817679558]


In [2]:
a, b, c = 0, 0, 0

print(a, b, c)

SyntaxError: illegal expression for augmented assignment (<ipython-input-2-b5e5c79f0a1a>, line 2)